### Monkey Patching меняет поведение функции динамически

In [17]:
class MyClass():
    def __init__(self, a, b):
        self.a = a
        self.b = b
    
    def add(self):
        return self.a + self.b

my = MyClass(3, 4)
print(my.add())
MyClass.speak = "OK"  #Monkey Patching
print(my.speak)

7
OK


### Можем сделать monkey patching с lambda

In [15]:
MyClass.speak2 = lambda self, message: "This is a message: {}".format(message)
print(my.speak2("My Message"))

This is a message: My Message


<b>На примере класса Fraction модуля fractions</b>

In [26]:
from fractions import Fraction

f = Fraction(2, 3)
print(f)
print(f.denominator)
print(f.numerator)

2/3
3
2


In [16]:
Fraction.speak = 100 #Monkey Patching
print(f.speak)

100


<b>Более полезное применение</b>

In [33]:
Fraction.is_integral = lambda self: self.denominator == 1
f1 = Fraction(2, 3)
f2 = Fraction(64, 8)
print(f1.is_integral())
print(f2.is_integral())

False
True


<b>Декорируем класс</b>

In [37]:
def dec_speak(cls):
    cls.speak = lambda self, message: "{0} says: {1}".format(
        self.__class__.__name__, message)  #cls.speak это метод класса, и вызываться будет соответственно
    return cls

"""cls.speak можно было бы прописать в классе, напр. так

class NewClass():
    def __init__(self):
        pass

    def speak(self):
        return "{0} says: {1}".format(self.__class__.__name__, message)

"""


Fraction = dec_speak(Fraction)
f1 = Fraction(2, 3)
print(f1.denominator)
print(f1.speak("Hello"))

class Person():
    pass

Person = dec_speak(Person)
p = Person()
p.speak("this is a message")

3
Fraction says: Hello


'Person says: this is a message'

### Debuger

In [40]:
from datetime import datetime, timezone


def info(self):
    result = []
    result.append("time: {}".format(datetime.now(timezone.utc)))
    result.append("Class: {}".format(self.__class__.__name__))
    result.append("id: {}".format(hex(id(self))))
    for k, v in vars(self).items():
        result.append("{}: {}".format(k, v))
    return result


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Person():
    def __init__(self, name, birth_year):
        self.name = name
        self.birth_year = birth_year

    # этот метод нигде не вызывается, написан просто для наглядности
    def say_hi():
        return "Hallo there!"


p = Person("Max", 1990)
print(p.debug())
# получается, что def info(self) - это метод класса, вынесенный из класса

['time: 2020-05-15 16:08:22.164286+00:00', 'Class: Person', 'id: 0x2478a62ddc8', 'name: Max', 'birth_year: 1990']


In [41]:
from datetime import datetime, timezone


def info(self):
    result = []
    result.append("time: {}".format(datetime.now(timezone.utc)))
    result.append("Class: {}".format(self.__class__.__name__))
    result.append("id: {}".format(hex(id(self))))
    for k, v in vars(self).items():
        result.append("{}: {}".format(k, v))
    return result


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Automobile():
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError("Speed connot exceed top_speed")
        else:
            self._speed = new_speed


favorite = Automobile("Frod", "Model T", 1908, 45)

print(favorite.debug())
favorite.speed = 40
print(favorite.debug())


['time: 2020-05-15 16:47:43.858763+00:00', 'Class: Automobile', 'id: 0x2478a657708', 'make: Frod', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 0']
['time: 2020-05-15 16:47:43.859766+00:00', 'Class: Automobile', 'id: 0x2478a657708', 'make: Frod', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 40']


### Усложним

In [1]:
def info(self):
    from datetime import datetime
    result = []
    result.append(f"time: {datetime.now()}")
    result.append(f"class: {self.__class__.__name__}")
    result.append("id: {}".format(hex(id(self))))
    for key, values in vars(self).items():
        result.append(f"{key}: {values}")
    return result


def debug_info(cls):
    cls.debug = info
    return cls


@debug_info
class Automobile():
    def __init__(self, make, model, year, top_speed):
        self.make = make
        self.model = model
        self.year = year
        self.top_speed = top_speed
        self._speed = 0

    @property
    def speed(self):
        return self._speed

    @speed.setter
    def speed(self, new_speed):
        if new_speed > self.top_speed:
            raise ValueError("Speed can not exceed top speed")

        else:
            self._speed = new_speed


auto = Automobile("Ford", "Model T", 1908, 45)
print(auto.debug())
auto.speed = 40
print(auto.debug())
print(auto.speed)

['time: 2020-05-17 21:09:21.478366', 'class: Automobile', 'id: 0x2a875b4b4c8', 'make: Ford', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 0']
['time: 2020-05-17 21:09:21.479361', 'class: Automobile', 'id: 0x2a875b4b4c8', 'make: Ford', 'model: Model T', 'year: 1908', 'top_speed: 45', '_speed: 40']
40


### Сравниваем два класса

In [2]:
from math import sqrt


class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return "Point({0}, {1})".format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if type(self) == type(other):
            return abs(self) < abs(other)
        else:
            return NotImplemented


a1 = Point(10, 20)
print(a1)
print(dir(a1))

Point(10, 20)
['__abs__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'x', 'y']


### Сделаем сравнение двух классов через monkey patching

In [17]:
from math import sqrt

class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 * self.y ** 2)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if type(self) == type(other):
            return abs(self) < abs(other)
        else:
            return NotImplemented


a = Point(10, 20)
b = Point(10, 2340)
print(abs(a))
print(a == b)
print(a >= b) #когда пробуем сравнить так, то получаем ошибку, т.к. метод "больше или равно" у нас не определён

200.0
False


TypeError: '>=' not supported between instances of 'Point' and 'Point'

<b>Добавим другие виды сравнений через monkey patching</b>

In [22]:
#обычно такие вещи как complete_ordering не делают, делаем только для примера
def complete_ordering(cls):
    if "__eq__" in dir(cls) and "__lt__" in dir(cls):
        cls.__le__ = lambda self, other: self < other or self == other
        cls.__gt__ = lambda self, other: not (
            self < other) and not (self == other)
        cls.__ge__ = lambda self, other: not (self < other)
    return cls


#чтобы работатло в классе нужно определить __eq__ и __lt__
@complete_ordering
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 + self.y ** 2)

    def __repr__(self):
        return "Point({0}, {1})".format(self.x, self.y)

    def __eq__(self, other):
        if isinstance(other, Point):
            return self.x == other.x and self.y == other.y
        else:
            return False

    def __lt__(self, other):
        if type(self) == type(other):
            return abs(self) < abs(other)
        else:
            return NotImplemented


p1, p2, p3 = Point(2, 3), Point(2, 3), Point(0, 0)
print(abs(p1))
print(p1 is p2)
print(p1 == p2)
print(p1 < p2)
print(p1 >= p2)
print(p1 != p2)
print(p1 <= p3) #теперь такие сравнения не ломаются

3.605551275463989
False
True
False
True
False
False


### В библиотеке Python уже есть функционал, кот. делает тоже самое, что и def complete_ordering

In [24]:
from math import sqrt
from functools import total_ordering

#чтобы работатло в классе нужно отпределить один из операторов: < > <= >=. Мы определили __lt__ (less than)
@total_ordering
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __abs__(self):
        return sqrt(self.x ** 2 * self.y ** 2)

    def __lt__(self, other):
        if type(self) == type(other):
            return abs(self) < abs(other)
        else:
            return NotImplemented


a = Point(10, 20)
b = Point(10, 2340)
print(abs(a))
print(a == b)
print(a < b)
print(a <= b)

200.0
False
True
True
